# Uncapacitated Facility Location (UFL)

Given a set of potential depots $N = \{1, \ldots, n\} $ and a set $M = \{1, \ldots, m\} $ of clients, suppose there is a fixed cost $ f_j $ associated with the use of depot $j $, and a transportation cost $c_{ij} $ if all of client $i$'s order is delivered from depot $j$. The problem is to decide which depots to open, and which depot serves each client so as to minimize the sum of the fixed and transportation costs. Note that this problem is similar to the covering problem, except for the addition of the variable transportation costs.

## Decision Variables

- $y_j = 1 $ if depot $ j $ is used (opened), and $ y_j = 0 $ otherwise (fixed cost/depot opening variable)
- $ x_{ij} $ is the fraction of the demand of client $ i $ satisfied from depot $ j $

## Constraints

1. Satisfaction of the demand of each client:

$$
\sum_{j=1}^{n} x_{ij} = 1 \quad \text{for } i = 1, \ldots, m.
$$

2. Linking constraints between \( x_{ij} \) and \( y_j \) variables:

$$
\sum_{i=1}^{m} x_{ij} \leq m y_j \quad \text{for } j = 1, \ldots, n.
$$

(Note: $ \sum_{i \in M} x_{ij} \leq m $ and we use this to formulate the fixed cost constraint)

3. Variable domain constraints:

$$
y_j \in \{0, 1\} \quad \text{for } j = 1, \ldots, n.
$$

$$
x_{ij} \geq 0 \quad \text{for } i = 1, \ldots, m, \quad j = 1, \ldots, n.
$$

## Objective Function

Minimize the sum of fixed costs and transportation costs:

$$
\min \sum_{j=1}^{n} f_j y_j + \sum_{i=1}^{m} \sum_{j=1}^{n} c_{ij} x_{ij}.
$$

## Complete MIP Formulation

$$
\begin{aligned}
& \min \sum_{j=1}^{n} f_j y_j + \sum_{i=1}^{m} \sum_{j=1}^{n} c_{ij} x_{ij} \\
& \text{subject to:} \\
& \quad \sum_{j=1}^{n} x_{ij} = 1, \quad i = 1, \ldots, m \\
& \quad \sum_{i=1}^{m} x_{ij} \leq m y_j, \quad j = 1, \ldots, n \\
& \quad y_j \in \{0, 1\}, \quad j = 1, \ldots, n \\
& \quad x_{ij} \geq 0, \quad i = 1, \ldots, m, \quad j = 1, \ldots, n
\end{aligned}
$$


In [1]:
!pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 42.1 MB/s eta 0:00:00


In [2]:
import gurobipy as gp
from gurobipy import GRB
import random

In [3]:
facilities = ["F1", "F2", "F3"]
customers = ["C1", "C2", "C3"]
fixed_costs = {"F1": 100, "F2": 120, "F3": 150}
transport_costs = {
    "F1": {"C1": 10, "C2": 20, "C3": 15},
    "F2": {"C1": 25, "C2": 30, "C3": 5},
    "F3": {"C1": 15, "C2": 10, "C3": 20},
}

model = gp.Model("FacilityLocation_Small")
# Variables
y = model.addVars(facilities, vtype=GRB.BINARY, name="Open")
x = model.addVars(customers, facilities, vtype=GRB.BINARY, name="Assign")
# Objective
model.setObjective(
    gp.quicksum(fixed_costs[f] * y[f] for f in facilities) +
    gp.quicksum(transport_costs[f][c] * x[c, f] for f in facilities for c in customers),
    GRB.MINIMIZE
)
# Constraints
for c in customers:
  model.addConstr(gp.quicksum(x[c, f] for f in facilities) == 1, name=f"Assign_{c}")
  for f in facilities:
    model.addConstr(x[c, f] <= y[f], name=f"Link_{c}_{f}")
model.optimize()

print("Opened facilities:")
for f in facilities:
  if y[f].X > 0.5:
    print(" ", f)
print("Assignments:")
for c in customers:
  for f in facilities:
    if x[c, f].X > 0.5:
      print(f"  Customer {c} → {f} (cost {transport_costs[f][c]})")
print("Total cost:", model.ObjVal, "\n")

Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 12 rows, 12 columns and 27 nonzeros
Model fingerprint: 0x107e3ce8
Variable types: 0 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 305.0000000
Presolve removed 5 rows and 5 columns
Presolve time: 0.00s
Presolved: 7 rows, 7 columns, 17 nonzeros
Found heuristic solution: objective 290.0000000
Variable types: 0 continuous, 7 integer (7 binary)
Found heuristic solution: objective 285.0000000

Root relaxation: objective 1.450000e+02, 5 iterations, 0.00 seconds (0.00 work units)

    Nodes 

In [4]:
random.seed(10)
facilities = [f"F{i+1}" for i in range(5)]
customers = [f"C{j+1}" for j in range(8)]
fixed_costs = {f: random.randint(50, 150) for f in facilities}
transport_costs = {f: {c: random.randint(5, 40) for c in customers} for f in facilities}
model = gp.Model("FacilityLocation_Random")
y = model.addVars(facilities, vtype=GRB.BINARY, name="Open")
x = model.addVars(customers, facilities, vtype=GRB.BINARY, name="Assign")
model.setObjective(
    gp.quicksum(fixed_costs[f] * y[f] for f in facilities) +
    gp.quicksum(transport_costs[f][c] * x[c, f] for f in facilities for c in customers),
    GRB.MINIMIZE
)
for c in customers:
  model.addConstr(gp.quicksum(x[c, f] for f in facilities) == 1, name=f"Assign_{c}")
  for f in facilities:
    model.addConstr(x[c, f] <= y[f], name=f"Link_{c}_{f}")
model.optimize()

print("Facilities fixed costs:", fixed_costs)
print("Opened facilities:")
for f in facilities:
  if y[f].X > 0.5:
    print(" ", f)

print("Assignments:")
for c in customers:
  for f in facilities:
    if x[c, f].X > 0.5:
      print(f"  Customer {c} → {f} (cost {transport_costs[f][c]})")
print("Total cost:", model.ObjVal)

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 48 rows, 45 columns and 120 nonzeros
Model fingerprint: 0x99869c9a
Variable types: 0 continuous, 45 integer (45 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 627.0000000
Presolve time: 0.00s
Presolved: 48 rows, 45 columns, 120 nonzeros
Variable types: 0 continuous, 45 integer (45 binary)

Root relaxation: objective 2.230000e+02, 29 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     223.0000000  223.00000 